In [1]:
%matplotlib notebook
from matplotlib import pyplot as plt
from google.colab import drive
import os
import random
import numpy as np
import pickle
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!python3 "/content/drive/My Drive/Evolution.AI---RN/more_clevr_state.py"

directory ./data already exists
building test datasets...
building train datasets...
building validation datasets...
saving datasets...
datasets saved at ./data/more-clevr_state.pickle


In [3]:
def load_data():
    print('loading data...')
    dirs = './data'
    filename = os.path.join(dirs,'more-clevr_state.pickle')
    with open(filename, 'rb') as f:
      train_datasets, test_datasets, val_datasets = pickle.load(f)
    rel_train = []
    rel_test = []
    rel_val = []
    norel_train = []
    norel_test = []
    norel_val = []
    print('processing data...')

    for img, relations, norelations in train_datasets:
        for qst,ans in zip(relations[0], relations[1]):
            rel_train.append((img,qst,ans))
        for qst,ans in zip(norelations[0], norelations[1]):
            norel_train.append((img,qst,ans))

    for img, relations, norelations in test_datasets:
        for qst,ans in zip(relations[0], relations[1]):
            rel_test.append((img,qst,ans))
        for qst,ans in zip(norelations[0], norelations[1]):
            norel_test.append((img,qst,ans))
            
    for img, relations, norelations in val_datasets:
        for qst,ans in zip(relations[0], relations[1]):
            rel_val.append((img,qst,ans))
        for qst,ans in zip(norelations[0], norelations[1]):
            norel_val.append((img,qst,ans))
    
    return (rel_train, rel_test,rel_val, norel_train, norel_test,norel_val)
def cvt_data_axis(data):
    img = [e[0] for e in data]
    qst = [e[1] for e in data]
    ans = [e[2] for e in data]
    return (img,qst,ans)
def split_data(data):
    Q1 = []#shape of object
    Q2 = []#query vertical position->yes/no
    Q3 = []#query horizontal position->yes/no
    Q4 = []#closest-to->rectangle/circle
    Q5 = []#furthest-from->rectangle/circle
    Q6 = []#count same color->1~6
    Q7 = []#if color2 is above the current object --->yes or no
    Q8 = []#if color2 is on the left to the current object --->yes or no
    random.shuffle(data)
    for ind,sample in enumerate(data):
        Q = sample[1]
        if Q[12] == 1: #non-rel questions
            if Q[14] == 1:
                Q1.append(sample)
            elif Q[15] == 1:
                Q2.append(sample)
            elif Q[16] == 1:
                Q3.append(sample)
        elif Q[13] == 1:#rel questions
            if Q[14] == 1:
                Q4.append(sample)
            elif Q[15] == 1:
                Q5.append(sample)
            elif Q[16] == 1:
                Q6.append(sample)
            elif Q[17] == 1:
                Q7.append(sample)
            elif Q[18] == 1:
                Q8.append(sample)

    Q1 = cvt_data_axis(Q1)
    Q2 = cvt_data_axis(Q2)
    Q3 = cvt_data_axis(Q3)
    Q4 = cvt_data_axis(Q4)
    Q5 = cvt_data_axis(Q5)
    Q6 = cvt_data_axis(Q6)
    Q7 = cvt_data_axis(Q7)
    Q8 = cvt_data_axis(Q8)
    return Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8

In [4]:
rel_train, rel_test,rel_val,norel_train, norel_test,norel_val = load_data()
_,_,_,Q4,Q5,Q6,Q7,Q8 = split_data(rel_test)

loading data...
processing data...


In [17]:
n=50 #index of question
objects = np.reshape(Q4[0][n],(6,10)).tolist()

my_obj = objects[Q4[1][n].tolist().index(1)][6:8]# coordinate of object's centre
dist_list = [((np.asarray(my_obj)-np.asarray(obj[6:8])) ** 2).sum() for obj in objects]
dist_list[dist_list.index(0)] = 999
closest = dist_list.index(min(dist_list))
if objects[closest][-2] == 1:
    answer = 2
else:
    answer = 3
print("checked result: "+str(answer))
print("dataset result: "+str(Q4[2][n]))

checked result: 2
dataset result: 2


In [6]:
objects

[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 44.0, 49.0, 1.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 30.0, 24.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 51.0, 23.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 13.0, 13.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 29.0, 61.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 57.0, 53.0, 1.0, 0.0]]

In [7]:
dist_list

[821.0, 999, 442.0, 410.0, 1370.0, 1570.0]

In [23]:
np.shape(Q8)

(3, 624)